In [2]:
import time
import sys
import winsound

from selenium import webdriver

from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from get_brand_list import get_brand_list
from get_clothes_info import get_clothes_info
from save_json import save_json
from load_json import load_json
from transformer import transform_datatype_and_soldout

In [3]:
# 1페이지에 존재하는 품목들을 가져오는 함수
def get_product_list():
    css = '#contents > div > div > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    product_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    products = product_list.find_elements_by_xpath(xpath) # 여기에 모든 element가 담김(리스트로)
    print(f'length: {len(products)}')
    return products

# 품목 카테고리 가져오는 함수
def get_product_category():
    """ex: OUTWEAR"""

    css = '#contents > div > div > div.xans-element-.xans-product.xans-product-headcategory.title > h2'
    product_category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_category = product_category.text
    print(product_category)
    return product_category

# 품목을 클릭하는 함수
def click_product(iter):
    css1 = '#contents > div > div > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    css2 = 'div > a'
    # 드라이버 재정의
    product_list = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css1)))
    products = product_list.find_elements_by_xpath(xpath)
    # url
    product_link = products[iter].find_element_by_css_selector(css2).get_attribute('href')
    print(f'link:{product_link}')
    # 박스 클릭
    driver.get(product_link)
    
    time.sleep(1)
    return product_link

# 품목 이름을 가져오는 함수
def get_product_name():
    css = '#contents > div:nth-child(1) > div.xans-element-.xans-product.xans-product-detail > div > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(1) > td > span'
    product_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_name = product_name.text.strip().split('\n')[1]
    print(f'name:{product_name}')
    return product_name

# 품목 브랜드를 가져오는 함수
def get_product_brand():
    """
    명시된 브랜드명을 그대로 가져옴
    우리 브랜드명에 맞춘 변환은 get_brand_list.py 함수에서 함
    실행은 get_clothes_info() 함수에서 받아서 변환
    """
    css = '#contents > div:nth-child(1) > div.xans-element-.xans-product.xans-product-detail > div > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(1) > td > span > b'
    product_brand = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_brand = product_brand.text[1:-1]
    print(f'brand:{product_brand}')
    return product_brand

# 품목 가격을 가져오는 함수
def get_product_price():
    css = '#span_product_price_text'
    product_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_price = product_price.text
    product_price = product_price.replace(",", '').replace(" ", "").replace("￦", '')
    print(f'price:{product_price}')
    return product_price

# 할인 가격을 가져오는 함수
def get_product_sale_price():
    css = '#span_product_coupon_dc_price'
    product_sale_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_sale_price = product_sale_price.text.split(' ')[0] # \금액
    product_sale_price = product_sale_price.replace(",", '').replace(" ", "").replace("￦", '')
    print(f'sale_price:{product_sale_price}')
    return product_sale_price

# 품목 사이즈를 가져오는 함수
def get_product_size():
    id = 'prdDetail'
    product_sizes = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, id))).get_attribute("innerHTML")
    product_sizes = product_sizes.strip().split('\n')

    for parse in product_sizes:
        if '가슴' in parse:
            type_ = '가슴'
            product_size = parse.split('가슴')[1].split(':')[1].strip()[:2]
            print(f'type: {type_}, size:{product_size}')
            return type_, product_size
        elif '허리' in parse:
            type_ = '허리'
            product_size = parse.split('허리')[1].split(':')[1].strip()[:2]
            print(f'type: {type_}, size:{product_size}')
            return type_, product_size
    type_ = '프리'
    product_size = '0'
    print(f'type: {type_}, size:{product_size}')
    return type_, product_size

# 썸네일 이미지를 가져오는 함수
def get_product_thumbnail():
    css = '#contents > div:nth-child(1) > div.xans-element-.xans-product.xans-product-detail > div > div.xans-element-.xans-product.xans-product-image.imgArea > div > div > img'
    product_thumbnail = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css))).get_attribute('src')
    print(f'thumbnail:{product_thumbnail}')
    return product_thumbnail

# 성별 구분하는 함수
def get_product_sex():
    css = '#contents > div:nth-child(1) > div.xans-element-.xans-product.xans-product-detail > div > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(1) > td > span'
    product_size = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_size = product_size.text.replace('[', '').replace(']','').split(' ')
    if 'MEN' in product_size: 
        sex = 'men'
    elif "WOMEN" in product_size: 
        sex = 'women'
    elif "UNISEX" in product_size:
        sex = 'unisex'
    print(f'gender:{sex}')
    return sex
    
# 품목 카테고리 구분하는 함수
def get_product_category_type(product_category, name):
    if '리버시블' in name.split(' ') or '세트' in name.split(' '):
        category = 'others'
    elif product_category == 'OUTWEAR':
        category ='outer'
    elif product_category in ['KNIT,CARDICAN', 'TOP', 'TOP(S/S)']:
        category = 'top'
    elif product_category in ['BOTTOM']:
        if '스커트' in name.split(' '):
            category = 'skirt'
        else:
            category = 'bottom'
    elif product_category == 'DRESS':
        category = 'dress'
    else:
        category = 'others'
    
    print(f'category:{category}')
    return category

# 품절 여부 확인하는 함수
def is_product_sold_out():
    
    css = '#contents > div:nth-child(1) > div.xans-element-.xans-product.xans-product-detail > div > div.infoArea > div.xans-element-.xans-product.xans-product-action > div.ec-base-button > a.first.displaynone'
    try:
        WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        print(f'품절 상품') # img src가 있으면 sold out 아이콘이 있는 거니까 품절
        return True
    except: # img src가 없으면 판매중
        print(f'판매중인 상품')
        return False

In [4]:
# # test
# driver = webdriver.Chrome(executable_path='chromedriver')
# store_name = '릴리즘'
# file_name = f'{store_name}.json' ### 바꿔 ###
# for i in range(1,21): ##### 바꿔 #####
#     URL = f'https://relizm.com/product/womenlist.html?cate_no=283&page={i}' ##### 바꿔 #####
#     driver.get(url=URL)
#     total_list = load_json(file_name)
#     product_list = get_product_list()
#     for iter in range(len(product_list)):
#             print('----------------{} page, {} -----------------'.format(i, iter+1))
#             # print(product_list[i].get_attribute('innerHTML'))

#             product_category_name = get_product_category()
#             product_link = click_product(iter)
#             try:
#                 product_name = get_product_name()
#                 product_brand_before = get_product_brand()
#                 product_price = get_product_price()
#                 product_sale_price = get_product_sale_price()
#                 product_type, product_size = get_product_size()
#                 product_thumbnail = get_product_thumbnail()
#                 product_gender = get_product_sex()
#                 product_category = get_product_category_type(product_category_name, product_name)
#                 is_sold_out = is_product_sold_out()
#                 product_brand_after = get_brand_list(product_brand_before)
#             except:
#                 time.sleep(0.5)
#                 # driver.close()
#                 break
            
#             json_ = get_clothes_info(store_name, product_sale_price, product_name, product_category, product_gender,
#     product_brand_after, product_price, product_thumbnail, product_link, is_sold_out, product_size, product_type)
#             total_list.append(json_)
#             break
#             time.sleep(0.5)
#             driver.get(url=URL)
#     # save_json(data=total_list, filename=file_name)
# winsound.Beep(440, 500)

In [8]:
# 바꿨는지 확인
check = input('바꾸셨나요?(y/n)')
if check == 'y':
    pass
else: # 종료
    sys.exit()
driver = webdriver.Chrome(executable_path='chromedriver')
store_name = '릴리즘'
file_name = f'{store_name}.json' ### 바꿔 ###
 
for i in range(6, 11): ##### 바꿔 #####
    URL = f'https://relizm.com/product/menlist.html?cate_no=287&page={i}' ##### 바꿔 #####
    driver.get(url=URL)
    total_list = load_json(file_name)
    product_list = get_product_list()
    for iter in range(len(product_list)):
        print('----------------{} page, {} -----------------'.format(i, iter+1))
        # print(product_list[i].get_attribute('innerHTML'))

        product_category_name = get_product_category()
        product_link = click_product(iter)
        try:
            product_name = get_product_name()
            product_brand_before = get_product_brand()
            product_price = get_product_price()
            product_sale_price = get_product_sale_price()
            product_type, product_size = get_product_size()
            product_thumbnail = get_product_thumbnail()
            product_gender = get_product_sex()
            product_category = get_product_category_type(product_category_name, product_name)
            is_sold_out = is_product_sold_out()
            product_brand_after = get_brand_list(product_brand_before)
        except:
            time.sleep(0.5)
            driver.get(url=URL)
            continue
        json_ = get_clothes_info(store_name, product_sale_price, product_name, product_category, product_gender,
 product_brand_after, product_price, product_thumbnail, product_link, is_sold_out, product_size, product_type)
        total_list.append(json_)
        time.sleep(0.5)
        driver.get(url=URL)
    save_json(data=total_list, filename=file_name)
    del total_list

winsound.Beep(440, 500)

length: 120
----------------1 page, 1 -----------------
OUTWEAR
link:https://relizm.com/product/detail.html?product_no=136908&cate_no=287&display_group=1
name:울 혼방 재킷
brand:SANRIKA
price:58000
sale_price:29000
type: 가슴, size:49
thumbnail:https://relizm.com/web/product/big/b181407.jpg
gender:women
category:outer
판매중인 상품
SANRIKA -> etc
----------------1 page, 2 -----------------
OUTWEAR
link:https://relizm.com/product/detail.html?product_no=136906&cate_no=287&display_group=1
name:코튼 혼방 오픈형 트위드 재킷
brand:UNKNOWN
price:44000
sale_price:22000
type: 가슴, size:44
thumbnail:https://relizm.com/web/product/big/b181405.jpg
gender:women
category:outer
판매중인 상품
UNKNOWN -> etc
----------------1 page, 3 -----------------
OUTWEAR
link:https://relizm.com/product/detail.html?product_no=136897&cate_no=287&display_group=1
name:폴리 베스트 재킷 세트
brand:SUNCA
price:48000
sale_price:24000
type: 가슴, size:45
thumbnail:https://relizm.com/web/product/big/b181395.jpg
gender:women
category:others
판매중인 상품
SUNCA -> etc
-----

In [ ]:
check_